In [104]:
import numpy as np
import numpy.linalg as LA

def sub_matrix(M, ids):
    dims = len(ids)
    N = np.zeros((dims, dims))
    for i in range(len(ids)):
        row = ids[i]
        for j in range(len(ids)):
            col = ids[j]
            N[i,j] = M[row,col]
    return N

def DPP(L,k):
    Y = []
    dim = L.shape[1]
    Z = list(range(dim))
    C = [[] for i in range(dim) ]
    D = [L[i,i] for i in range(dim)]
    j = np.argmax(D)
    Y.append(j)
    Z.remove(j)
    
    dj = []
    dj.append(D[j])
    print("Det:{},items:{}".format(np.prod(dj),Y))

    while len(Z)>0 and len(Y)<k:
        max_pos = -1
        max_value = None
        for i in Z:
            ei = (L[j,i] - np.dot(C[j],C[i])) / np.sqrt(D[j])
            C[i].append(ei)
            D[i] = D[i] - ei * ei
            if max_pos == -1 or max_value < D[i]:
                max_pos = i
                max_value = D[i]
        
        j = max_pos
        dj.append(D[j])
        Y.append(j)
        print("Det:{},items:{}".format(np.prod(dj),Y))     
        Z.remove(j)
    return Y

def greedy(L,k):
    dim = L.shape[1]
    Y = []
    Z = list(range(dim))
    X = []
    while len(Z)>0 and len(Y)<k:
        max_pos = -1
        max_value = None
        id2det = dict()
        for i in Z:
            candidate_list = X
            candidate_list.append(i)
            matrix = sub_matrix(L,candidate_list)
            det_value = LA.det(matrix)
            id2det[i] = det_value
            if max_pos == -1 or max_value < det_value:
                max_pos = i
                max_value = det_value
            candidate_list.pop()
        j = max_pos

        #print("list:{},max:{},det:{}".format(X,max_value,id2det))
        X.append(j)
        Y.append(j)
        Z.remove(j)
    return Y 
            
        
np.random.seed(134)
x = np.random.rand(10,10) * 2

X = np.mat(np.dot(x.T, x))
print(X)
#print(LA.det(X))
#E,V = LA.eigh(X)
#print("E:\n{}\nV:\n{}".format(E,V))
#print(sub_matrix(X,[0,3,2]))
print(DPP(X,6))
print(greedy(X,6))

[[18.81303201 16.52360258 10.07824282 10.39288506 13.51458745 14.57054024
  14.34696292 15.6578766   7.41406334 10.52262805]
 [16.52360258 16.65870168 10.31031494  9.27183847 13.65820598 14.47099459
  12.4853748  14.28239892  6.18406959 11.97814485]
 [10.07824282 10.31031494 12.04002251  6.77720511 11.24398495 12.39377982
   7.57619823  9.08523933  6.38329328  8.92045608]
 [10.39288506  9.27183847  6.77720511  9.95060595  7.78970681 10.85914719
   8.86607138  9.65875928  4.33626451  7.68772019]
 [13.51458745 13.65820598 11.24398495  7.78970681 16.49003452 14.36283806
   8.64052887 11.16743179  6.26857821 11.87558669]
 [14.57054024 14.47099459 12.39377982 10.85914719 14.36283806 19.11865135
  11.34154017 15.48986444  7.63203563 14.27531867]
 [14.34696292 12.4853748   7.57619823  8.86607138  8.64052887 11.34154017
  12.78025525 13.28316735  5.88186171  7.56898217]
 [15.6578766  14.28239892  9.08523933  9.65875928 11.16743179 15.48986444
  13.28316735 16.40643083  7.62505603 11.96260793]


In [87]:
def get_det(D, Z):
    value = 1.0
    for i in Z:
        value *= D[i]
    return value

def cholesky(A):
    ret = np.linalg.cholesky(A)
    return ret

def sub_matrix_cholesky(Y,ids):
    matrix = sub_matrix(Y, ids)
    ret = cholesky(matrix)
    return ret

def greedy_one_step(L,Y,Z):
    print("greedy one step y:{0},z:{1}".format(Y,Z))
    id2det = dict()
    for i in Z:
        Y.append(i)
        matrix = sub_matrix(L,Y)
        det_value = LA.det(matrix)
        id2det[i] = det_value
        Y.pop()
    idx,value = max(id2det.items(),key=lambda x:x[1])
    print("list:{0},max_id:{1},max_value:{2}\n det:{3}".format(Y,idx,value,id2det))
    


In [106]:
def dpp_sliding_window(L, window_size, k):
    dim = L.shape[1]
    V = np.zeros((window_size,window_size))
    C = [[] for i in range(dim)]
    D = np.array([L[i,i] for i in range(dim)])
    Y = []
    Z = [i for i in range(dim)]
    #greedy_one_step(L,Y[-window_size+1:],Z)
    j = np.argmax(D)
    Y.append(j)
    Z.remove(j)
    
    while len(Z)>0 and len(Y)< k:
        if len(Y) > window_size:
            V1 = np.zeros((window_size,window_size))
            V1[:-1,:-1] = V[0:,0:]
            V = V1

        row = len(Y)-1
        if row >= (window_size-1):
            row = window_size - 1
        #print(C,row)
        for i in range(row):
            V[row,i] = C[j][i]
        V[row,row] = np.sqrt(D[j]) 
        #print("row:{1}\nV:\n{0}".format(V,row))
        print("V:\n{0}, det:{1}".format(V, LA.det(V)**2)) 
        m2 = sub_matrix_cholesky(L,Y[-window_size:])
        #print("cholesky deco:{0},det:{1}".format(m2,LA.det(m2)**2))
        for i in Z:
            ei = (L[j,i] - np.dot(C[j],C[i])) / np.sqrt(D[j])
            C[i].append(ei)
            D[i] = D[i] - ei * ei
        if len(Y) >= window_size:
            v = V[1:,0].copy()
            V = V[1:,1:]
            A = []
            C1 = []
            for i in range(len(C)):
                if i in Z:
                    A.append(C[i][0])
                    C1.append(list(C[i][1:]))
                else:
                    A.append(None)
                    C1.append([])

            #print("C1:{},C:{}".format(C1,C))
            C = C1
            for l in range(window_size-1):
                t2 = V[l,l]**2 + v[l] ** 2
                t = np.sqrt(t2)
                V[l+1:,l] = (V[l+1:,l]*V[l,l] + v[l+1:]*v[l]) / t
                v[l+1:] = (v[l+1:] * t - V[l+1:,l] * v[l]) / V[l,l]
                for i in Z:
                    C[i][l] = (C[i][l]*V[l,l] + A[i]*v[l]) / t
                    A[i] = (A[i]*t - C[i][l]*v[l]) / V[l,l]
                V[l,l] = t
            for i in Z:
                D[i] = D[i] + A[i]*A[i]           

        kv = list(zip(Z,D[Z]))
        max_pos,max_value = max(kv, key=lambda x:x[1])      
        #print(max_pos, max_value, kv)
        #greedy_one_step(L,Y[-window_size+1:],Z)
        j = max_pos
        Y.append(j)
        Z.remove(j)

        print("solution:{}".format(Y))
                  
dpp_sliding_window(X,4,8)        

V:
[[4.372488 0.       0.       0.      ]
 [0.       0.       0.       0.      ]
 [0.       0.       0.       0.      ]
 [0.       0.       0.       0.      ]], det:0.0
solution:[5, 0]
V:
[[4.372488   0.         0.         0.        ]
 [3.33232252 2.77644713 0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]], det:0.0
solution:[5, 0, 4]
V:
[[4.372488   0.         0.         0.        ]
 [3.33232252 2.77644713 0.         0.        ]
 [3.28482046 0.92510541 2.20094731 0.        ]
 [0.         0.         0.         0.        ]], det:0.0
solution:[5, 0, 4, 9]
V:
[[ 4.372488    0.          0.          0.        ]
 [ 3.33232252  2.77644713  0.          0.        ]
 [ 3.28482046  0.92510541  2.20094731  0.        ]
 [ 3.26480454 -0.12849286  0.57709668  1.94354385]], det:2696.77085213
solution:[5, 0, 4, 9, 2]
V:
[[4.33739922 0.         0.         0.        ]
 [3.11582743 2.6041609  0.         0.        ]
 [2.42602249 1.65754708

In [97]:
np.random.seed(134)
x = np.random.rand(10,10) * 2
X = np.mat(np.dot(x.T, x))
m,n = X.shape
np.savetxt("a.txt", X, delimiter='\t')